In [1]:
import pandas as pd
import numpy as np
import random as rd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import math

In [2]:
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./My lovely TA-5c13e90a75eb.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '13oItYj_NLkMlyD5T5PuAsdUtDeDtWHWWcKocHuCwIA0'
book = gc.open_by_key(spreadsheet_key)

In [4]:
def cekNektar(W,x):
    return ((W[0] + W[1]*x) / (1+W[2]*x+(W[3]**2)))

def newForager(scouts,Pxy,BSF,nScouts):
    Pzy = scouts[rd.randint(0,nScouts-1)]
    while (Pzy==Pxy):
        Pzy = rd.randint(0,nScouts-1)
    return Pxy+rd.uniform(-1,1)*(Pxy-Pzy)

def initialization(W,j,k,constraint):
    nEmployedBees = j
    nScouts = k
    nektar,onlookerBees,Bees = [],[],[]
    totalNektar = 0
    allScouts,allNectar = [],[] 
    for i in range(nScouts):
        jumlahNectar,scouts = [],[]
        for j in range(5):
            well = W[j]
            batasAtas = well[4]
            x = rd.uniform(0,batasAtas)
            y = cekNektar(well,x)
            scouts.append(x)
            jumlahNectar.append([y,x])
        while (sum(scouts)>constraint):
            for j in range(5):
                well = W[j]
                batasAtas = well[4]
                x = rd.uniform(0,batasAtas)
                y = cekNektar(well,x)
                scouts[j] = x
                jumlahNectar[j] =[y,x]
        allScouts.append(scouts)
        allNectar.append(jumlahNectar)
    return allScouts,allNectar

In [5]:
W = np.loadtxt('well.txt',delimiter=' ')
d = {'Posisi' : [], 'Jumlah Nektar' : []}

In [71]:
nScouts = 100
nEmployedBees = 60
constraint = 1000
for angka in range(1):
    Scouts,allNectar = initialization(W,nEmployedBees,nScouts,constraint)
    for abc in range(200):
        jumlahNektar = []
        for i in range(5):
            jumlahNektar.append([sub[i][1] for sub in allNectar])

            totalNektar = []
        for i in range(5):
            totalNektar.append(sum(jumlahNektar[i]))

        onlookerBees = []
        for i in range(nScouts): 
            asd = []
            for j in range(5):
                asd.append([allNectar[i][j][1]/totalNektar[j], allNectar[i][j][1]])
            onlookerBees.append(asd)

        nilaiProbs = []
        for i in range(nScouts):
            x = 0
            for j in range(5):
                x += onlookerBees[i][j][0]
            nilaiProbs.append([x,i])

        nilaiProbs.sort(reverse = True)    
        nilaiProbs = nilaiProbs[:nEmployedBees]

        Bees = []
        for i in range(nEmployedBees):
            Bees.append(onlookerBees[nilaiProbs[i][1]])
            
        for i in range(nEmployedBees):
            for j in range(5):
                Bees[i][j].append(cekNektar(W[j],Bees[i][j][1]))

        temp = []
        for i in range(5):
            temp.append([[sub[i][1],sub[i][2]] for sub in Bees])

        bestSoFar = []
        for i in range(5):
            best = temp[i][0]
            for j in range(nEmployedBees):
                if (best[1] < temp[i][j][1]):
                    best = temp[i][j]
            bestSoFar.append(best[0])

        for i in range(nScouts):
            for j in range(5):
                temp = [sub[j] for sub in Scouts]
                newPos = newForager(temp,Scouts[i][j],bestSoFar[j],nScouts)
                batasAtas = W[j][4]
                if (newPos<0 or newPos>batasAtas):
                    newPos = 0
                Scouts[i][j] = newPos
            while (sum(Scouts[i])>constraint):
                for j in range(5):
                    batasAtas = W[j][4]
                    temp = [sub[j] for sub in Scouts]
                    newPos = newForager(temp,Scouts[i][j],bestSoFar[j],nScouts)
                    if (newPos<0 or newPos>batasAtas):
                        newPos = 0
                    Scouts[i][j] = newPos
        best,beeTerbaik = 0,[]
        for i in Bees:
            total = 0
            for j in i:
                total += j[2]
            if total>best:
                best,beeTerbaik = total,i
                
    d['Jumlah Nektar'] = [sub[2] for sub in beeTerbaik]
    d['Jumlah Nektar'].append(sum([sub[2] for sub in beeTerbaik]))
    d['Posisi'] = [sub[1] for sub in beeTerbaik]
    d['Posisi'].append(sum([sub[1] for sub in beeTerbaik]))
    df = pd.DataFrame(data=d)
    wks_name = '1000 '+str(angka)
    d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
    print ('jalaan',angka)

SSLError: [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)